In [1]:
import cv2
import itertools
import numpy as np
from time import time
import mediapipe as mp
import matplotlib.pyplot as plt
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def getSize(image, face_landmarks, INDEXES):
    '''
    This function calculates the height and width of a face part utilizing its landmarks.
    Args:
        image:          The image of person(s) whose face part size is to be calculated.
        face_landmarks: The detected face landmarks of the person whose face part size is to 
                        be calculated.
        INDEXES:        The indexes of the face part landmarks, whose size is to be calculated.
    Returns:
        width:     The calculated width of the face part of the face whose landmarks were passed.
        height:    The calculated height of the face part of the face whose landmarks were passed.
        landmarks: An array of landmarks of the face part whose size is calculated.
    '''
    
    # Retrieve the height and width of the image.
    image_height, image_width, _ = image.shape
    
    # Initialize a list to store the landmarks of the face part.
    landmarks = []
    
    # Iterate over the indexes of the landmarks of the face part. 
    for INDEX in INDEXES:
        
        # Append the landmark into the list.
        landmarks.append([int(face_landmarks.landmark[INDEX].x * image_width),
                               int(face_landmarks.landmark[INDEX].y * image_height)])
    
    # Calculate the width and height of the face part.
    # TODO: Transform it to take into account the sheearing --> find the 3D boundinx box and project 
    # the closest face to the screen
    _, _, width, height = cv2.boundingRect(np.array(landmarks))
    
    # Convert the list of landmarks of the face part into a numpy array.
    landmarks = np.array(landmarks)
    
    # Return the calculated width height and the landmarks of the face part.
    return width, height, landmarks

In [3]:
def isOpen(image, face_mesh_results, face_part, threshold=5):
    '''
    This function checks whether the eye or mouth of the person(s) is open, 
    utilizing its facial landmarks.
    Args:
        image:             The image of person(s) whose an eye or mouth is to be checked.
        face_mesh_results: The output of the facial landmarks detection on the image.
        face_part:         The name of the face part that is required to check: MOUTH, RIGHT EYE, LEFT EYE
        threshold:         The threshold value used to check the isOpen condition.
    Returns:
        status:       A dictionary containing isOpen statuses of the face part of all the 
                      detected faces.  
    '''
    
    # Retrieve the height and width of the image.
    image_height, image_width, _ = image.shape

    
    # Create a dictionary to store the isOpen status of the face part of all the detected faces.
    status={}
    
    # Check if the face part is mouth.
    if face_part == 'MOUTH':
        
        # Get the indexes of the mouth.
        INDEXES = mp_face_mesh.FACEMESH_LIPS
        
    # Check if the face part is left eye.    
    elif face_part == 'LEFT EYE':
        
        # Get the indexes of the left eye.
        INDEXES = mp_face_mesh.FACEMESH_LEFT_EYE
    
    # Check if the face part is right eye.    
    elif face_part == 'RIGHT EYE':
        
        # Get the indexes of the right eye.
        INDEXES = mp_face_mesh.FACEMESH_RIGHT_EYE 
    
    # Otherwise return nothing.
    else:
        return
    
    # Convert the indexes of the landmarks of the face part into a list.
    # TODO: Rewrite it in a more natural way OR undestand it + Is it necessaray to create it as a list?
    INDEXES_LIST = set(list(itertools.chain(*INDEXES)))
    
    # Iterate over the found faces.
    for face_no, face_landmarks in enumerate(face_mesh_results.multi_face_landmarks):
        
         # Get the height of the face part.
        _, height, _ = getSize(image, face_landmarks, INDEXES_LIST)
        
         # Get the height of the whole face.
        face_oval = set(list(itertools.chain(*mp_face_mesh.FACEMESH_FACE_OVAL)))
        _, face_height, _ = getSize(image, face_landmarks, face_oval)
        
        # Check if the face part is open.
        if (height/face_height)*100 > threshold:
            
            # Set status of the face part to open.
            status[face_no] = 'OPEN'
        
        # Otherwise.
        else:
            # Set status of the face part to close.
            status[face_no] = 'CLOSE'
    
    # Otherwise
    else:
        
        # Return the output image and the isOpen statuses of the face part of each detected face.
        return status

In [4]:
def overlay(image, filter_img, face_landmarks, face_part, INDEXES):
    '''
    This function will overlay a filter image over a face part of a person in the image/frame.
    Args:
        image:          The image of a person on which the filter image will be overlayed.
        filter_img:     The filter image that is needed to be overlayed on the image of the person.
        face_landmarks: The facial landmarks of the person in the image.
        face_part:      The name of the face part on which the filter image will be overlayed.
        INDEXES:        The indexes of landmarks of the face part.
        display:        A boolean value that is if set to true the function displays 
                        the annotated image and returns nothing.
    Returns:
        annotated_image: The image with the overlayed filter on the top of the specified face part.
    '''
    
    # Create a copy of the image to overlay filter image on.
    annotated_image = image.copy()
    
    # Errors can come when it resizes the filter image to a too small or a too large size .
    # So use a try block to avoid application crashing.
    try:
            
        # Get the width and height of filter image.
        filter_img_height, filter_img_width, _  = filter_img.shape


        # Check if the face part is mouth.
        if face_part == 'MOUTH':
            # Get the height of the face part on which we will overlay the filter image.
            _, face_part_height, landmarks = getSize(image, face_landmarks, INDEXES)

            # Specify the height to which the filter image is required to be resized.
            # 2.5 can be changed depending on the size of the filter we want
            # This allows the filter to be bigger/smaller depending on both the size of the eye and the size of the aperture
            # of the eye
            required_height = int(face_part_height*1.5)

            # Resize the filter image to the required height, while keeping the aspect ratio constant. 
            resized_filter_img = cv2.resize(filter_img, (int(filter_img_width*(required_height/filter_img_height)),
                                                         required_height))

            # Get the new width and height of filter image.
            filter_img_height, filter_img_width, _  = resized_filter_img.shape
            
            # Calculate the center of the face part.
            center = landmarks.mean(axis=0).astype("int")

            # Calculate the location where the mouth filter will be placed.  
            # Location is the position of the upper left corner of the filter image
            location = (int(center[0] - filter_img_width / 2), int(center[1]))

        # Otherwise if the face part is an eye.
        elif face_part == 'LEFT EYE' or face_part == 'RIGHT EYE':
            
            # Get the height of the face part on which we will overlay the filter image.
            _, face_part_height, landmarks = getSize(image, face_landmarks, INDEXES)

            # Specify the height to which the filter image is required to be resized.
            # 2.5 can be changed depending on the size of the filter we want
            # This allows the filter to be bigger/smaller depending on both the size of the eye and the size of the aperture
            # of the eye
            required_height = int(face_part_height*2.5)

            # Resize the filter image to the required height, while keeping the aspect ratio constant. 
            resized_filter_img = cv2.resize(filter_img, (int(filter_img_width*(required_height/filter_img_height)),
                                                         required_height))

            # Get the new width and height of filter image.
            filter_img_height, filter_img_width, _  = resized_filter_img.shape
            
            # Calculate the center of the face part.
            center = landmarks.mean(axis=0).astype("int")

            # Calculate the location where the eye filter image will be placed.  
            # Location is the position of the upper left corner of the filter image
            location = (int(center[0]-filter_img_width/2), int(center[1]-filter_img_height/2))
            
        elif face_part == 'FOREHEAD':
                        
            # Get the height of the face part on which we will overlay the filter image.
            face_part_width, _, landmarks = getSize(image, face_landmarks, INDEXES)
            
            # Specify the height to which the filter image is required to be resized.
            # 2.5 can be changed depending on the size of the filter we want
            # This allows the filter to be bigger/smaller depending on both the size of the eye and the size of the aperture
            # of the eye
            required_width = int(face_part_width*2.5)

            # Resize the filter image to the required height, while keeping the aspect ratio constant. 
            resized_filter_img = cv2.resize(filter_img, (required_width,
                                                         int(filter_img_height*(required_width/filter_img_width))))

            # Get the new width and height of filter image.
            filter_img_height, filter_img_width, _  = resized_filter_img.shape
            
            # Calculate the center of the face part.
            center = landmarks.mean(axis=0).astype("int")

            # Calculate the location where the eye filter image will be placed.  
            # Location is the position of the upper left corner of the filter image
            location = (int(center[0]-filter_img_width/2), int(center[1]-filter_img_height))
            
            

        # Retrieve the region of interest from the image where the filter image will be placed.
        ROI = image[location[1]: location[1] + filter_img_height,
                    location[0]: location[0] + filter_img_width]
        
        # Convert the image to grayscale and apply the threshold to get the invert mask of the filter image.
        # We obtain a filter_img_mask with 255 in black area (pixel < 25) and 0 in other areas
        # This is used to put the black background in transparent
        _, filter_img_mask = cv2.threshold(cv2.cvtColor(resized_filter_img, cv2.COLOR_BGR2GRAY),
                                           25, 255, cv2.THRESH_BINARY_INV)

        # Perform Bitwise-AND operation. 
        # Black-out the area of the filter in ROI
        resultant_image = cv2.bitwise_and(ROI, ROI, mask=filter_img_mask)

        # Add the resultant image and the resized filter image.
        # This will update the pixel values of the resultant image at the indexes where 
        # pixel values are zero, to the pixel values of the filter image.
        resultant_image = cv2.add(resultant_image, resized_filter_img)

        # Update the image's region of interest with resultant image.
        annotated_image[location[1]: location[1] + filter_img_height,
                        location[0]: location[0] + filter_img_width] = resultant_image
            
    # Catch and handle the error(s).
    except Exception as e:
        pass
            
    # Return the annotated image.
    return annotated_image

In [5]:
# Initialize the mediapipe face mesh class.
mp_face_mesh = mp.solutions.face_mesh

# Setup the face landmarks function for videos.
# TODO: Here works for only one face --> Make it work for multiple faces (see overlay call in main function)
face_mesh_videos = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, 
                                         min_detection_confidence=0.5,min_tracking_confidence=0.3)

In [6]:
# Initialize the mediapipe drawing class.
mp_drawing = mp.solutions.drawing_utils

# Initialize the mediapipe drawing styles class.
mp_drawing_styles = mp.solutions.drawing_styles

In [7]:
# Initialize the VideoCapture object to read from the webcam.
camera_video = cv2.VideoCapture(0)

# Set camera resolution
camera_video.set(3,1280)
camera_video.set(4,960)

# Create named window for resizing purposes.
cv2.namedWindow('Face Filter', cv2.WINDOW_NORMAL)

# Read the left and right eyes images.
# TODO: Create an interactive window to let the user select the wanted filters
left_eye = cv2.imread('data/left_eye_cupcake.png')
right_eye = cv2.imread('data/right_eye_cupcake.png')

crown = cv2.imread('data/crown.png')

# Initialize the VideoCapture object to read from the smoke animation video stored in the disk.
# TODO: Same as for images --> Create a dictionary to store all images and videos?
animation = cv2.VideoCapture('data/rainbow_animation1.mp4')

# Set the smoke animation video frame counter to zero.
animation_frame_counter = 0

# Iterate until the webcam is accessed successfully.
while camera_video.isOpened():
    
    # Read a frame.
    ok, frame = camera_video.read()
    
    # Check if frame is not read properly then continue to the next iteration to read
    # the next frame.
    if not ok:
        continue
        
    # Read a frame from smoke animation video
    _, animation_frame = animation.read()
    
    # Increment the smoke animation video frame counter.
    animation_frame_counter += 1
    
    # Check if the current frame is the last frame of the animation video.
    if animation_frame_counter == animation.get(cv2.CAP_PROP_FRAME_COUNT):     
        
        # Set the current frame position to first frame to restart the video.
        animation.set(cv2.CAP_PROP_POS_FRAMES, 0)
        
        # Set the animation video frame counter to zero.
        animation_frame_counter = 0
    
    # Flip the frame horizontally for natural (selfie-view) visualization.
    frame = cv2.flip(frame, 1)
    
    # Perform the facial landmarks detection on the image, after converting it into RGB format.
    # TODO: check why we have to put ::-1
    face_mesh_results = face_mesh_videos.process(frame[:,:,::-1])
    
    output_image = frame.copy()
    
    # Check if facial landmarks are found.
    if face_mesh_results.multi_face_landmarks:        
        # Get the mouth isOpen status of the person in the frame.
        mouth_status = isOpen(frame, face_mesh_results, 'MOUTH', 
                                     threshold=15)
        
        # Get the left eye isOpen status of the person in the frame.
        left_eye_status = isOpen(frame, face_mesh_results, 'LEFT EYE', 
                                        threshold=4.5)
        
        # Get the right eye isOpen status of the person in the frame.
        right_eye_status = isOpen(frame, face_mesh_results, 'RIGHT EYE', 
                                         threshold=4.5)
        
        # Iterate over the found faces.
        for face_num, face_landmarks in enumerate(face_mesh_results.multi_face_landmarks):
            
            # Check if the left eye of the face is open.
            if left_eye_status[face_num] == 'OPEN':
                # Convert the indexes of the landmarks of the face part into a list.
                # TODO: Rewrite it in a more natural way OR undestand it + Is it necessaray to create it as a list?
                left_eye_landmarks = set(list(itertools.chain(*mp_face_mesh.FACEMESH_LEFT_EYE)))
                
                # Overlay the left eye image on the frame at the appropriate location.
                # TODO: I don't know if this will work with multiple faces because of mp_face_mesh.FACEMESH_LEFT_EYE 
                # (same for the 2 other) because it considers all meshes and not only one mesh --> Adapt it
                # to multiple meshes if it doesn't work
                frame = overlay(frame, left_eye, face_landmarks,
                                'LEFT EYE', left_eye_landmarks)
            
            # Check if the right eye of the face is open.
            if right_eye_status[face_num] == 'OPEN':
                
                # Convert the indexes of the landmarks of the face part into a list.
                # TODO: Rewrite it in a more natural way OR undestand it + Is it necessaray to create it as a list?
                right_eye_landmarks = set(list(itertools.chain(*mp_face_mesh.FACEMESH_RIGHT_EYE)))
                
                # Overlay the right eye image on the frame at the appropriate location.
                frame = overlay(frame, right_eye, face_landmarks,
                                'RIGHT EYE', right_eye_landmarks)
                            
            lips_landmarks = set(list(itertools.chain(*mp_face_mesh.FACEMESH_LIPS)))
            # Check if the mouth of the face is open.
            if mouth_status[face_num] == 'OPEN':
                
                # Convert the indexes of the landmarks of the face part into a list.
                # TODO: Rewrite it in a more natural way OR undestand it + Is it necessaray to create it as a list?
                lips_landmarks = set(list(itertools.chain(*mp_face_mesh.FACEMESH_LIPS)))
                
                # Overlay the smoke animation on the frame at the appropriate location.
                frame = overlay(frame, animation_frame, face_landmarks, 
                                'MOUTH', lips_landmarks)
                
            forehead_landmarks = [103, 67, 109, 10, 338, 297, 332]
            frame = overlay(frame, crown, face_landmarks, 'FOREHEAD', forehead_landmarks)
            
    
    # Display the frame.
    cv2.imshow('Face Filter', frame)
    
    # Wait for 1ms. If a key is pressed, retreive the ASCII code of the key.
    k = cv2.waitKey(1) & 0xFF    
    
    # Check if 'ESC' is pressed and break the loop.
    if(k == 27):
        break

# Release the VideoCapture Object and close the windows.                  
camera_video.release()
cv2.destroyAllWindows()